# MAPseq Data Processing

In this Jupyter notebook, we will guide you through the steps for turning raw sequencing reads into a viral barcode matrix, which you can then analyze yourself or in conjunction with BARseq data, as we will do in this course.

All the code is Python, using standard libraries and idioms. But some of the key functionality is provided by the Pandas library, which itself is built on NumPy. To learn more about them, and the Python language visit: 

- https://www.python.org/
- https://pandas.pydata.org/ 
- https://numpy.org/
# Imports

In [ ]:
# Built-in python libraries
import logging
import os
import sys
from configparser import ConfigParser

# Data science libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask
import dask.dataframe as dd
from dask.dataframe import from_pandas

# For handling barcode tags or other fields with letters and numbers. 
from natsort import natsorted

# Allowing to run our custom libraries from git area. 
gitpath=os.path.expanduser("~/git/mapseq-processing")
sys.path.append(gitpath)
from mapseq.core import *
from mapseq.barcode import *
from mapseq.utils import *
from mapseq.bowtie import *
from mapseq.stats import *

gitpath=os.path.expanduser("~/git/mapseq-analysis")
sys.path.append(gitpath)
from msanalysis.analysis import * 
print("Done")

# Overview

The overall goal is to take the paired-end RNA sequencing FASTQ data and produce a correct matrix of viral barcodes and their projections to dissected areas. This is implemented as a pipeline of functions, with the first consuming the FASTQs and the others consuming the output of the previous step, until the final step produces the matrices.  

![title](files/mapseq-processing-schema-api.png)

# Configuration, logging, and paths for analysis
We need to provide paths to the two fastq read files (either fastq or fastq.gz), the path to the a standard Python configuration file, and a standard sample information Excel spreadsheet. We also have a convenience method to package up the FASTQ file pairs.  

This is an example sample info spreadsheet. 

![title](files/sampleinfo-xlsx.png)

The Site information column should indicate whether the dissected area is a target or injection, and additionally whether it is a negative (biological control) or control (water). 
Region and Brain should be obvious. Note that the region labels will be used (if available) for the default plots.  

The RT primer barcode table is included in the software, as 

<code>~/git/mapseq-processing/etc/barcodes_v2.txt</code>

By default, functions in the pipeline take their parameters from a single configuration file, included in the distribution <code>~/git/mapseq-processing/etc/mapseq.conf</code>. If you want to perform processing with parameters other than the defaults, then copy the config in the distrubtion, edit it, and use it instead. For some functions, parameters can be overridden in the function call itself.  

Here are a couple example sections of the configuration file. We will revisit it for each function in more detail. 

![title](img/mapseq-config.png)

A configuration file simply has sections, within which there are key-value pairs. These values can be retrieved by the ConfigParser interface.  


In [ ]:
#cp = get_default_config()
configfile = os.path.expanduser('~/mapseq/M205/M205.mapseq.conf')
cp = ConfigParser()
cp.read(configfile)
logging.getLogger().setLevel(logging.INFO)
sampleinfo = os.path.expanduser('~/mapseq/M205/M205.sampleinfo.xlsx')
bcfile = os.path.expanduser( cp.get('barcodes','ssifile') )
project_id = cp.get('project','project_id')

infilelist = [
    os.path.expanduser('~/mapseq/M205/fastq/M205_HZ_S1_R1_001.fastq.gz'),
    os.path.expanduser('~/mapseq/M205/fastq/M205_HZ_S1_R2_001.fastq.gz')
          ]
infiles = package_pairs(infilelist)
outdir = os.path.expanduser('~/mapseq/M205')
print(f"For {project_id}:\nconfig={configfile}\nbcfile={bcfile}\ninfiles={infiles}\noutdir={outdir}\nDone")

# Load Sample Information

We load the information from the Excel spreadsheet into a Pandas dataframe. What you see should correspond to what was in the Excel file. 
If it does not, check the formatting of the fields in the spreadsheet (leading/trailing spaces, incorrect capitalization, etc.). The loading code expects the column headers to be exactly as shown here, no blank rows, and no extraneous info below the main table.   


In [ ]:
sampdf = load_sample_info(sampleinfo, cp=cp)
sampdf

# process_fastq_pairs()

Here is an example of the first few lines of a FASTQ-formatted data file. 

![title](files/fastq-format.png)

Information about each read uses four lines of the file. The first line is the sequence ID, often containing info about the experiment, sequencing hardware, and run-specific metadata.  
The second line is the actual sequence. 
The third line may have optional information. 
The fourth line has encoded quality estimates for each of the bases in line two.  

We are only interested in the sequence line. 

Note that some raw data may come in multiple pairs of files, and is often compressed (with gzip). The program can read multiple pairs as long as they sort properly alphabetically, and it correctly handles compressed or uncompressed files. This function pulls out the sequence lines from the first read FASTQ, and joins them with the corresponding sequence from the seconds read FASTQ.

The function pulls out all the sequence lines, splits the sequence by fields, and generates a read-oriented table in a form that is easy to manipulate. Each row of the dataframe is a single read. 

![title](files/fastq-schema.png)

Run the cell. It will call the function and display the resulting table. For our data this should take about a minute. As it runs, it will display progress messages per 1000000 sequences.    


In [ ]:
# handle all the input. usually takes ~2 minutes
# M205.htna25
logging.getLogger().setLevel(logging.DEBUG)
rdf = process_fastq_pairs(infiles, outdir=outdir, cp=cp)
logging.getLogger().setLevel(logging.INFO)
rdf

Note that the number of rows represents the total number of raw reads. At this point we have retained the initial 52nt full read.
Below is additional Pandas code that shows examples of how further information can be extracted at this point. Feel free to edit and re-run. Whatever the last variable or call is will be displayed.   

### aggregate_reads_dd()
Each row represents a read, but many of these rows are identical (or they should be, if the sequencing read depth is sufficient). 
So we aggregate the reads and set a read_count column. This greatly reduces the dataframe size. We are using Dask under the hood to do this, because most real datasets will exceed system memory.  

Dask requires a dataframe in its own format, so we convert it. 

This step also drops reads with only a single copy. If you want to retain all, set min_reads=1. 

In [ ]:
adf = aggregate_reads(rdf, outdir=outdir, cp=cp)
adf

Note how the row count has dropped signficantly--from 17M to 4M. 

# split_fields()  and filter_fields()

Now we are going to filter the reads in order to remove rows that have 'N' characters, which are nucleotides the sequencing process was unable to resolve. We will also remove sequences with long runs of the same nucleotide (>7) since we know these result in unreliable sequences. 

We will also now split the overall sequence into MAPseq-specific fields, and drop the full sequence column (since we don't need it anymore) 

This function should only take a few seconds...

In [ ]:
sdf = split_fields(adf.copy(), column='sequence', drop=True, cp=cp )
sdf

In [ ]:
fdf = filter_fields(sdf.copy(), cp=cp)
fdf

Below is an area to again play with data exploration. 

In [ ]:
# Compare libtag diversity after filtering with that above. 
fdf['libtag'].value_counts()

This is a good time to be aware that many of the functions in the pipeline fill in statistics and QC information in a file named <code>stats.\<datestring\>.json</code> where the datestring is the time the Jupyter session was initialized. These stats files are intended to be both human and machine readable, so they can be used to make reports, or plots. 

Here is an example of the contents of the stats file at this point of the processing:

![title](files/stats-fastq.png)

In [ ]:
fdf['read_count'].max()
fdf['read_count'].median()
fdf['read_count'].describe()
fdf['vbc_read'].value_counts().describe()

# process_make_readtable_pd()

Now that we have the fields of interest, we can fill in, for convenience, the information determined by the SSI barcode, identify the libtags, and identify spike-in sequences. As part of that process, we will also identify reads which have errors such that that information cannot be filled in, and remove them. 

This is where we calculate an estimate of the template switching rate. The evidence for this is when we see an L1 libtag in a read with a valid target SSI. Information needed to calculate the rate is in the stats entry for the readtable function.

As part of this we will automatically generate read count frequency plots so that we can confirm that our read count threshold (2 by default) are reasonable. Frequency plots are done separately for target and injection, since those are sequenced in different pools. In the raw plot, we include an **estimated_threshold**. This is a value calculated as the threshold that retains 90% of the total data.  

This step should only take a few seconds...


In [ ]:
rtdf = process_make_readtable_pd( fdf.copy(),
                                   sampdf,
                                   bcfile=bcfile, 
                                   outdir=outdir, 
                                   cp=cp)
rtdf

Note that data from the sample information spreadsheet has now been integrated into the read table. The type value has been set from the libtag and the spike sequence. The label, rtprimer number, and brain id has been set from matching the SSI sequence. 

Since this is a small dataset, and read depth is thus low, a read threshold of 2 is reasonable. 

Below is a cell to play with...

In [ ]:
# Let's get an idea of the number of reals and spikes by target area
rtdf.groupby(['label','type'], observed=True ).agg( { 'umi':'nunique', 'read_count':'sum'}).reset_index()


# align_collapse()

As the MAPseq virus replicates in the cell, it is prone to a fairly high error rate. If we didn't account for this, we might attribute viral barcodes that came from a single cell to multiple cells. So we are going to find all the sets of viral barcodes (vbc_read) column that are within a Hamming distance of 3 (i.e. all sequences that can be made identical with 3 or less single nucleotide edits). For each of those sets, we will set the vbc_read sequence to the same sequence. This schematic illustrates the process.  

![title](files/align-collapse.png)

Note that the number of rows has not changed, and each row still represents one read. Even after collapse we retain the read_count for the rows, so at any point we can threshold on that value. 

We will perform the search by doing an all x all alignment using bowtie. We parse the bowtie output to create an edge graph. We use Tarjan's algorithm to find all the components for a given graph. Then we collapse the vbc_read sequences to the most common sequence variant in the component.  

This is the most memory and CPU-intensive function in the pipeline, and can take a long time for large datasets. For the workshop the dataset is relatively small, so it should complete in 3-5 minutes. 

In [ ]:
cdf = align_collapse_pd(rtdf.copy(), 
                       outdir=outdir, 
                       cp=cp)
cdf

Again, note that the total number of rows is the same. The vbc_read column now contains the "fixed" sequence. 

We can look at the new entry in the stats file for deeper insight. 

![title](files/stats-collapse.png)

We can see that of ~16M VBC sequences, there were only 715k unique vbc_read sequences. Component numbers can also be informative. n_components is the total number of Hamming sets. n_multi_components is the number of those sets that had more than one distinct sequence in them (meaning there was a mutation). So the large majority of Hamming components consist of sequence variants.  

The cell below is for exploration...

In [ ]:
# Compare diversity of read values after collapse to the one above before colllapse
# Why did the mean go down?
cdf['vbc_read'].value_counts().describe()

# process_make_vbctable_pd()

So far the data tables have been read-oriented (each row is a read). Now we are ready to move toward a viral barcode-oriented format, with each row representing a viral barcode, which represents a single neuron. And this is the step where we will threshold on the read_count value we checked above. 

After data thresholding and cleaning, and dropping some redundant columns, this function aggregates on the viral barcode and sums the UMI count. This function takes less than a minute.  


In [ ]:
vdf = process_make_vbctable_pd(cdf.copy(),
                               outdir=outdir,
                               cp=cp)
vdf

Note that the number of rows has now dropped from ~16M to ~350K, and we have per-barcode umi_count and (aggregate) read_count for each UMI. Recall that the vbc_read_col column has duplicate entries, but they should differ in other columns. 

Below is a cell to explore...

In [ ]:
# 

# process_filter_vbctable()

We now have the full viral barcode table, with UMI counts for real and spike-in sequences.
We previously thresholded the data for read_counts, and removed reads with invalid or missing values. This function prepares the data so that the matrix generation code can be as simple as possible. So here we will establish minimum UMI counts for inclusion in the final matrices. 

This experiment does not have injection barcodes from MAPseq (they will come from BARseq). If it did, this is where we would only include barcodes that appear in both an injection area and at least one target. 

In [ ]:
vfdf = process_filter_vbctable( vdf, 
                                target_min_umi = 2, 
                                outdir=outdir,
                                cp=cp)

# process_make_matrices_pd()

For each brain, we pivot the real VBCs against the SSI/labelled site to create a 2D matrix, with the value being the UMI count for that VBC in that site.
We do the same operation for real VBCs filtered by the thresholds and requirements as defined in the config file. 
We do the same operation for the spike-in barcodes. 
Within each target area, we then normalize the filtered real UMI count by the total spike-in UMI count for that area: 

![title](files/normalization.png)

The results are pre-brain normalized barcode matrices, labelled by brain. E.g. YW144.nbcm.tsv

The file names as follows:
- rbcm   = raw real barcode matrix
- fbcmdf = filtered (real) barcode matrix
- sbcm   = spike-in matrix
- nbcm   = normalized barcode matrix
- scbcm  = scaled normalized barcode matrix. 

The scale normalized matrix forces all values to 0.0 - 1.0 for usage in heatmap plots that require scaled values. 

This function should run in less than 30 seconds. Since this function doesn't produce a single dataframe, we print a less-pretty version of the normalized matrices. All the matrices are saved as TSV files in the out directory.  


In [ ]:
(real_dict, spike_dict, norm_dict) = process_make_matrices(vdf,
                                                           sampdf=sampdf,  
                                                           outdir=outdir,
                                                           cp=cp)
for bid in norm_dict.keys():
    print(f'brain={bid}:\n{norm_dict[bid]}\n')

As matrices, this data is a bit harder to infer anything by eye, but is in a form useful for plotting. 

Below is a cell for exploration...

In [ ]:
nbcm = norm_dict['YW143']
nbcm
nbcm['BC5'].max()

# Output files

Much more data is produced than is reflected in this notebook. To further investigate you can examine the output directory to open those files. 

In [ ]:
flist = os.listdir(outdir)
flist.sort()
flist

# Further data analysis

For simple data validation, we find it useful to create binarized plots, organized hierarchically. 
The y-axis is all the viral barcodes, while the x-axis is the target areas. For each barcode, the target area is given color if any UMIs are present. We have hierarchically arranged the VBCs such that all VBCs that project to any combination of target areas are grouped together. The vertical height of the color block in any target area reflects the number of projections. And blocks arraned horizontally in line with one another represent neurons with projections to multiple target areas. This allows the identification of projection motifs.

Since in this case we have two brains with the same target areas, obvious patterns across them are likely to reflect biological reality. 


In [ ]:
infile = f'{outdir}/YW143.nbcm.tsv'
nbmdf = load_mapseq_df(infile)
nbmdf
g = get_plot_binarized(nbmdf, expid=f'{project_id}:YW143') 

In [ ]:
infile = f'{outdir}/YW144.nbcm.tsv'
nbmdf2 = load_mapseq_df(infile)
nbmdf2
g = get_plot_binarized(nbmdf2, expid=f'{project_id}:YW144') 